# Challenge Set 9
## Part II: Baseball Data

*Introductory - Intermediate level SQL*

--

Please complete this exercise via SQLalchemy and Jupyter notebook.

We will be working with the Lahman baseball data we uploaded to your AWS instance in class. 


1. What was the total spent on salaries by each team, each year?

2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

3. Who has played the most all star games?

4. Which school has generated the most distinct players? *Hint:* Create new table from 'CollegePlaying.csv'.

5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

6. What is the distribution of debut months? *Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

7. What is the effect of table join order on mean salary for the players listed in the main (master) table? *Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.


In [1]:
import sqlalchemy as db
import pandas as pd

Create a connection to the AWS server instance running postgres:

In [2]:
params = {
    'host': '34.217.89.93',
    'user': 'ubuntu',
    'port': 5432
}
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/baseball'
engine = db.create_engine(connection_string)
connection = engine.connect()
metadata = db.MetaData()
engine.table_names()

['allstarfull', 'salaries', 'schools', 'teams', 'fielding', 'master']

Load the teams table if its not already loaded, then get a link to it:

In [3]:
if not engine.dialect.has_table(engine,'teams'):
    teams_df = pd.read_csv('baseballdata/teams.csv')
    teams_df.to_sql('teams', con=engine)
    del teams_df
else:
    print('teams Database table already exists!  teams.drop(engine) and rerun to recreate.')
teams = db.Table('teams', metadata, autoload=True, autoload_with=engine)
print('# of teams Table fields: ',len(teams.columns.keys()))

teams Database table already exists!  teams.drop(engine) and rerun to recreate.
# of teams Table fields:  49


In [4]:
if not engine.dialect.has_table(engine,'salaries'):
    salaries_df = pd.read_csv('baseballdata/salaries.csv')
    salaries_df.to_sql('salaries', con=engine)
    del salaries_df
else:
    print('salaries Database table already exists!  salaries.drop(engine) and rerun to recreate.')
salaries = db.Table('salaries', metadata, autoload=True, autoload_with=engine)
print('# of salaries Table fields: ',len(salaries.columns.keys()))
salaries.columns.keys()

salaries Database table already exists!  salaries.drop(engine) and rerun to recreate.
# of salaries Table fields:  5


['yearid', 'teamid', 'lgid', 'playerid', 'salary']

## What was the total spent on salaries by each team, each year?

Using information from these sites:  
* https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91
* https://docs.sqlalchemy.org/en/13/orm/session_basics.html#what-does-the-session-do
* https://blog.miguelgrinberg.com/post/nested-queries-with-sqlalchemy-orm

In [5]:
from sqlalchemy.orm import sessionmaker
# create a configured "Session" class
Session = sessionmaker(bind=engine)
# create a Session
session = Session()

In [6]:
subq = session.query(teams.c.yearID,teams.c.lgID,teams.c.teamID,teams.c.name).subquery()

In [7]:
# results = connection.execute(subq)
# df = pd.DataFrame(results)
# df.columns = results.keys()
# df[df.yearID>1984]

In [8]:
subq2 = session.query(salaries.c.yearid,\
                   salaries.c.lgid,\
                   salaries.c.teamid,\
                   db.func.sum(salaries.c.salary).label('Salaries')).\
                   group_by(salaries.c.yearid,\
                                salaries.c.lgid,\
                               salaries.c.teamid).order_by(salaries.c.yearid,\
                                                           salaries.c.lgid,\
                                                           salaries.c.teamid).subquery()

In [9]:
# results = connection.execute(subq2)
# df = pd.DataFrame(results)
# df.columns = results.keys()
# df

In [10]:
query = db.select([subq2, subq])
query = query.select_from(subq2.join(subq, db.and_(subq2.c.yearid==subq.c.yearID,\
                           subq2.c.lgid==subq.c.lgID,\
                           subq2.c.teamid==subq.c.teamID)))
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df[['yearid','lgid','name','Salaries']].head(5)


,yearid,lgid,name,Salaries
0,1985,AL,Toronto Blue Jays,8812550.0
1,1985,AL,New York Yankees,14238204.0
2,1985,AL,Detroit Tigers,10348143.0
3,1985,AL,Baltimore Orioles,11560712.0
4,1985,AL,Boston Red Sox,10897560.0


In [11]:
session.close()

## What is the first and last year played for each player? 
Hint: Create a new table from 'Fielding.csv'.

In [12]:
if not engine.dialect.has_table(engine,'fielding'):
    fielding_df = pd.read_csv('baseballdata/Fielding.csv')
    fielding_df.to_sql('fielding', con=engine)
    del fielding_df
else:
    print('fielding Database table already exists!  fielding.drop(engine) and rerun to recreate.')
fielding = db.Table('fielding', metadata, autoload=True, autoload_with=engine)
print('# of fielding Table fields: ',len(fielding.columns.keys()))

fielding Database table already exists!  fielding.drop(engine) and rerun to recreate.
# of fielding Table fields:  19


In [13]:
if not engine.dialect.has_table(engine,'master'):
    master_df = pd.read_csv('baseballdata/master.csv')
    master_df.to_sql('master', con=engine)
    del master_df
else:
    print('master Database table already exists!  master.drop(engine) and rerun to recreate.')
master = db.Table('master', metadata, autoload=True, autoload_with=engine)
print('# of master Table fields: ',len(master.columns.keys()))


master Database table already exists!  master.drop(engine) and rerun to recreate.
# of master Table fields:  25


The next two cells use the db.select method to get the two queries that need to be joined, but I can't figure out how to do the join.

In [14]:
query = db.select([fielding,master])
query = query.select_from(fielding.join(master, fielding.c.playerID==master.c.playerID))
# results = connection.execute(query).fetchall()
# df = pd.DataFrame(results)
# df.columns = results[0].keys()
# df.head(5)

In [15]:
query = db.select([fielding.c.playerID, \
                  db.func.min(fielding.c.yearID).label('FirstYearPlayed'), \
                  db.func.max(fielding.c.yearID).label('LastYearPlayed')]).\
        group_by(fielding.c.playerID)
# results = connection.execute(query)
# df = pd.DataFrame(results)
# df.columns = results.keys()
# print(df.info())
# df.head()

Using the Session method gives the answer:

In [17]:
# create a configured "Session" class
Session = sessionmaker(bind=engine)
# create a Session
session = Session()       
names = session.query(master.c.playerID,master.c.nameFirst,master.c.nameLast).subquery()
playRange = session.query(fielding.c.playerID.label('playerID'), \
                          db.func.min(fielding.c.yearID).label('FirstYearPlayed'), \
                          db.func.max(fielding.c.yearID).label('LastYearPlayed')).\
            group_by(fielding.c.playerID).subquery()
query = db.select([playRange, names])
query = query.select_from(playRange.join(names, playRange.c.playerID==names.c.playerID))
results = connection.execute(query).fetchall()
session.close()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df[['nameFirst','nameLast','FirstYearPlayed','LastYearPlayed']].head(5)

,nameFirst,nameLast,FirstYearPlayed,LastYearPlayed
0,David,Aardsma,2004,2013
1,Hank,Aaron,1954,1976
2,Tommie,Aaron,1962,1971
3,Don,Aase,1977,1990
4,Andy,Abad,2001,2003


## Who has played the most all star games?

In [31]:
AllstarFull = db.Table('allstarfull', metadata, autoload=True, autoload_with=engine)
AllstarFull.columns.keys()

['playerid',
 'yearid',
 'gamenum',
 'gameid',
 'teamid',
 'lgid',
 'gp',
 'startingpos']

In [33]:
# create a configured "Session" class
Session = sessionmaker(bind=engine)
# create a Session
session = Session()       
names = session.query(master.c.playerID,master.c.nameFirst,master.c.nameLast).subquery()
allStarGames = session.query(AllstarFull.c.playerid.label('playerID'), \
                          db.func.count(AllstarFull.c.gameid).label('NumAllstarGames')).\
            group_by(AllstarFull.c.playerid).subquery()
query = db.select([allStarGames, names])
query = query.select_from(AllstarFull.join(names, AllstarFull.c.playerid==names.c.playerID))
results = connection.execute(query).fetchall()
session.close()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df[['nameFirst','nameLast','NumAllstarGames']].head(5)

,nameFirst,nameLast,NumAllstarGames
0,Lefty,Gomez,1
1,Lefty,Gomez,1
2,Lefty,Gomez,2
3,Lefty,Gomez,2
4,Lefty,Gomez,2
